### Uploading the packaged egg file to remote hdfs

In [7]:
# Push the egg file to user HDFS directory in the cluster
import os
import dsx_core_utils
dsx_core_utils.upload_hdfs_file(
    source_path=os.environ['DSX_PROJECT_DIR']+'/SpamFilter/dist/SpamFilter-1.0-py2.7.egg', 
    target_path="/user/user1/SpamFilter-1.0-py2.7.egg",
    webhdfsurl="https://zinc1.fyre.ibm.com:8443/gateway/jalv-126-master-1/webhdfs/v1")

upload success


### Uploading the datatset to remote cluster

In [8]:
# Push the dataset to user HDFS directory in the cluster
import dsx_core_utils
dsx_core_utils.upload_hdfs_file(
    source_path=os.environ['DSX_PROJECT_DIR']+'/datasets/SMSSpamCollection.csv', 
    target_path="/user/user1/SMSSpamCollection.csv",
    webhdfsurl="https://zinc1.fyre.ibm.com:8443/gateway/jalv-126-master-1/webhdfs/v1")

upload success


### Connecting to remote spark through DSX-HI

In [9]:
%load_ext sparkmagic.magics
from dsx_core_utils import proxy_util,dsxhi_util
proxy_util.configure_proxy_livy()

dsxhi_util.list_livy_endpoints()

The sparkmagic.magics extension is already loaded. To reload it, use:
  %reload_ext sparkmagic.magics
success configuring sparkmagic livy.
['https://miniedge-hd.fyre.ibm.com:8443/gateway/jalv-126-master-1/livy2/v1', 'https://zinc1.fyre.ibm.com:8443/gateway/jalv-126-master-1/livy/v1', 'https://zinc1.fyre.ibm.com:8443/gateway/jalv-126-master-1/livy2/v1']


In [4]:
%manage_spark

Added endpoint https://zinc1.fyre.ibm.com:8443/gateway/jalv-126-master-1/livy2/v1
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
54,application_1525285700946_0046,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


### Distributing the egg file to remote spark cluster

In [5]:
%%spark
sc.addPyFile("hdfs:///user/user1/SpamFilter-1.0-py2.7.egg")

### Invoke the LRModel custom method on remote cluster

In [6]:
%%spark
import SpamFilter
import sys
import os

from pyspark import SparkContext
from pyspark.sql import SparkSession

# Import libraries from the deployed egg
from SpamFilter import DataPipeline, LRModel

# Building the Pipeline
pipeline = DataPipeline().pipeline()

# Read the file from HDFS
filename = "/user/user1/SMSSpamCollection.csv"

# Call the method
LRModel().execute(sc,filename,pipeline)

Model Accuracy: 91.51
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   41|
|  0.0|       1.0|    5|
|  1.0|       0.0|    4|
|  0.0|       0.0|   56|
+-----+----------+-----+